In [1]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from minisom import MiniSom
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
from joblib import Parallel, delayed
import psutil
import warnings
import tracemalloc

In [24]:
# Funzione per convertire l'uso della memoria in unità appropriate
def format_memory_usage(mem_usage):
    if mem_usage >= 1024 ** 3:
        return f"{mem_usage / (1024 ** 3):.2f} GB"
    else:
        return f"{mem_usage / (1024 ** 2):.2f} MB"

In [25]:
# Funzione per classificare i dati utilizzando il SOM
def classify2(som, data, training_data, training_labels):
    from collections import Counter
    winmap = som.labels_map(training_data, training_labels)
    default_class = Counter(training_labels).most_common(1)[0][0]
    result = []
    for sample in data:
        winner = som.winner(sample)
        if winner in winmap:
            result.append(winmap[winner].most_common(1)[0][0])
        else:
            result.append(default_class)
    return result


In [26]:
# Function to monitor memory usage during a function
def monitor_memory_usage(func, *args, **kwargs):
    tracemalloc.start()
    result = func(*args, **kwargs)
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    return result, peak

In [27]:
# Funzione per il training e l'evaluation con monitoraggio della memoria
def train_and_evaluate(train_idx, test_idx, X, y, som_shape, sigma, learning_rate):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    som = MiniSom(som_shape[0], som_shape[1], X_train.shape[1], sigma=sigma, learning_rate=learning_rate)

    start_train = time.time()
    _, training_mem_usage = monitor_memory_usage(som.train_random, X_train, X_train.shape[0])
    end_train = time.time()

    start_eval = time.time()
    y_pred, eval_mem_usage = monitor_memory_usage(classify2, som, X_test, X_train, y_train)
    end_eval = time.time()

    report = classification_report(y_test, y_pred, digits=4, output_dict=True, zero_division=0)

    training_time = end_train - start_train
    evaluating_time = end_eval - start_eval
    total_mem_usage = training_mem_usage + eval_mem_usage

    return report, training_time, training_mem_usage, evaluating_time, eval_mem_usage, total_mem_usage

In [3]:
# Inizializzazione del dataset e dei parametri
DATA_PATH = "/users/mik2002/som/dataset-processed CIC-IDS-2018/"
FILES = ["processed_bot_dataset.csv", "processed_dos_dataset.csv","processed_infiltration_dataset.csv","processed_Ddos_dataset.csv", "processed_bruteforce_dataset.csv"]
df_dataset = pd.concat([pd.read_csv(DATA_PATH + file) for file in FILES], ignore_index=True)

display(df_dataset["Label"].value_counts())
print("Distribution:")
display(df_dataset["Label"].value_counts(normalize=True))

/tmp/ipykernel_212052/1234980745.py:4: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62) have mixed types. Specify dtype option on import or set low_memory=False.
  df_dataset = pd.concat([pd.read_csv(DATA_PATH + file) for file in FILES], ignore_index=True)


Label
Benign          10967059
DDos              576191
Dos               507715
Bot               282310
Infiltration      161060
Bruteforce        156668
Label                  1
Name: count, dtype: int64

Distribution:


Label
Benign          8.668924e-01
DDos            4.554508e-02
Dos             4.013239e-02
Bot             2.231522e-02
Infiltration    1.273101e-02
Bruteforce      1.238384e-02
Label           7.904511e-08
Name: proportion, dtype: float64

In [4]:
# Parametri le SOM
som_sizes = [(20, 20)]
learning_rate = 1.2
sigma = 1.5
sampling_fractions = [0.1]

In [30]:
results = []

# Wrapper function for parallelism
def parallel_train_and_evaluate(train_idx, test_idx, som_shape, data, target, sigma, learning_rate):
    report, training_time, training_mem_usage, evaluating_time, eval_mem_usage, total_mem_usage = train_and_evaluate(train_idx, test_idx, data, target, som_shape, sigma, learning_rate)
    return report, training_time, training_mem_usage, evaluating_time, eval_mem_usage, total_mem_usage

def display_report_and_times(all_reports, all_training_times, all_training_mem_usages, all_evaluating_times, all_eval_mem_usages, all_total_mem_usages):
    print("\nAll Reports and Times for this SOM Shape:")
    
    for i, (report, train_time, train_mem_usage, eval_time, eval_mem_usage, total_mem_usage) in enumerate(zip(all_reports, all_training_times, all_training_mem_usages, all_evaluating_times, all_eval_mem_usages, all_total_mem_usages)):
        print(f"\nReport for Fold {i+1}:")
        for label, metrics in report.items():
            if isinstance(metrics, dict):
                print(f"\nLabel {label}:")
                for metric_name, metric_value in metrics.items():
                    print(f"  {metric_name.capitalize()}: {metric_value:.4f}")
            else:
                print(f"\n{label.capitalize()}: {metrics:.4f}")
        print(f"\nTraining Time: {train_time:.4f} seconds")
        print(f"Training Memory Usage: {format_memory_usage(train_mem_usage)}")
        print(f"Evaluating Time: {eval_time:.4f} seconds")
        print(f"Evaluating Memory Usage: {format_memory_usage(eval_mem_usage)}")
        print(f"Total Memory Usage: {format_memory_usage(total_mem_usage)}")

# KFold initialization
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Loop over different sampling fractions
for frac in sampling_fractions:
    print(f"Evaluating for sampling fraction: {frac}")
    
    # Perform stratified random sampling maintaining class proportions
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", DeprecationWarning)
        sampled_df = df_dataset.groupby("Label", group_keys=False).apply(lambda x: x.sample(frac=frac, random_state=42))
    sampled_df["Label"] = sampled_df["Label"].map({"Benign": 0, "DDos": 1, "Dos": 2, "Bot": 3, "Bruteforce": 4})

    scaler = MinMaxScaler()
    data = scaler.fit_transform(sampled_df.iloc[:, :-1])
    target = sampled_df.iloc[:, -1].values

    # Loop over different SOM sizes
    for som_shape in som_sizes:
        print(f"Evaluating for som_shape: {som_shape}")
        # Split dataset with K-Fold and parallelism
        parallel_results = Parallel(n_jobs=-1)(delayed(parallel_train_and_evaluate)(train_idx, test_idx, som_shape, data, target, sigma, learning_rate) for train_idx, test_idx in kf.split(data))
        
        reports, training_times, training_mem_usages, evaluating_times, eval_mem_usages, total_mem_usages = zip(*parallel_results)
        
        # Aggregate metrics from reports
        class_metrics = {}
        for report in reports:
            for key, metrics in report.items():
                if key.isdigit() or key in ['macro avg', 'weighted avg']:
                    if key not in class_metrics:
                        class_metrics[key] = {m: [] for m in metrics.keys() if m != 'support'}
                    for metric, value in metrics.items():
                        if metric != 'support':
                            class_metrics[key][metric].append(value)

        # Calculate averages and prepare formatted output
        averages = {key: {metric: np.mean(values) for metric, values in metrics.items()} for key, metrics in class_metrics.items()}

        # Calculate average training and evaluation times and memory usage
        avg_training_time = np.mean(training_times)
        avg_training_mem_usage = np.mean(training_mem_usages)
        avg_evaluating_time = np.mean(evaluating_times)
        avg_eval_mem_usage = np.mean(eval_mem_usages)
        avg_total_mem_usage = np.mean(total_mem_usages)

        results.append((frac, som_shape, averages, avg_training_time, avg_training_mem_usage, avg_evaluating_time, avg_eval_mem_usage, avg_total_mem_usage, reports, training_times, training_mem_usages, evaluating_times, eval_mem_usages, total_mem_usages))

# Display results
print("\nAverage Metrics Across All Folds:")
for result in results:
    frac, shape, metrics, avg_training_time, avg_training_mem_usage, avg_evaluating_time, avg_eval_mem_usage, avg_total_mem_usage, all_reports, all_training_times, all_training_mem_usages, all_evaluating_times, all_eval_mem_usages, all_total_mem_usages = result
    print("****************************")
    print(f"\nSampling Fraction: {frac}")
    print(f"SOM Shape: {shape}".upper())
    for class_id, class_metrics in metrics.items():
        print(f"\nClass {class_id}:")
        for metric, value in class_metrics.items():
            print(f"{metric}: {value:.4f}")

    # Display average training and evaluation times and memory usage
    minutes_train, seconds_train = divmod(avg_training_time, 60)
    minutes_eval, seconds_eval = divmod(avg_evaluating_time, 60)
    print(f"Average Training Time: {int(minutes_train)} minutes and {seconds_train:.2f} seconds")
    print(f"Average Training Memory Usage: {format_memory_usage(avg_training_mem_usage)}")
    print(f"Average Evaluating Time: {int(minutes_eval)} minutes and {seconds_eval:.2f} seconds")
    print(f"Average Evaluating Memory Usage: {format_memory_usage(avg_eval_mem_usage)}")
    print(f"Average Total Memory Usage: {format_memory_usage(avg_total_mem_usage)}")
    print("---------------------------------------")
    display_report_and_times(all_reports, all_training_times, all_training_mem_usages, all_evaluating_times, all_eval_mem_usages, all_total_mem_usages)

Evaluating for sampling fraction: 0.1


Evaluating for som_shape: (20, 20)

Average Metrics Across All Folds:
****************************

Sampling Fraction: 0.1
SOM SHAPE: (20, 20)

Class 0:
precision: 0.9953
recall: 0.9911
f1-score: 0.9932

Class 1:
precision: 0.9769
recall: 0.9904
f1-score: 0.9836

Class 2:
precision: 0.9931
recall: 0.9442
f1-score: 0.9680

Class 3:
precision: 0.9128
recall: 0.9376
f1-score: 0.9243

Class 4:
precision: 0.8398
recall: 0.9982
f1-score: 0.9121

Class macro avg:
precision: 0.9436
recall: 0.9723
f1-score: 0.9562

Class weighted avg:
precision: 0.9861
recall: 0.9852
f1-score: 0.9854
Average Training Time: 2 minutes and 35.27 seconds
Average Training Memory Usage: 7.92 MB
Average Evaluating Time: 1 minutes and 18.51 seconds
Average Evaluating Memory Usage: 16.60 MB
Average Total Memory Usage: 24.52 MB
---------------------------------------

All Reports and Times for this SOM Shape:

Report for Fold 1:

Label 0:
  Precision: 0.9961
  Recall: 0.9952
  F1-score: 0.9956
  Support: 99336.0000

Labe

In [6]:
sampled_df = df_dataset.groupby("Label", group_keys=False).apply(lambda x: x.sample(frac=0.5, random_state=42))
sampled_df["Label"] = sampled_df["Label"].map({"Benign": 0, "DDos": 1, "Dos": 2, "Bot": 3, "Bruteforce": 4, "Infiltration"})

/tmp/ipykernel_212052/2249145038.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df_dataset.groupby("Label", group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))


In [8]:
sampled_df["Label"].value_counts()

Label
0.0    1096706
1.0      57619
2.0      50772
3.0      28231
4.0      15667
Name: count, dtype: int64

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = scaler.fit_transform(sampled_df.iloc[:, :-1])
target = sampled_df.iloc[:, -1].values

In [12]:
# split data in Training and Testing set
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(data, target,test_size=0.2)
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing  set has {} samples.".format(X_test.shape[0]))

Training set has 1012080 samples.
Testing  set has 253021 samples.


In [13]:
# since this in supervised classification not unsupervised, this function chooses the class of
# the BMU to be the final class of each instance
def classify(som, data):
    """Classifies each sample in data in one of the classes definited
    using the method labels_map.
    Returns a list of the same length of data where the i-th element
    is the class assigned to data[i].
    """
    winmap = som.labels_map(X_train, y_train)
    default_class = np.sum(list(winmap.values())).most_common()[0][0]
    result = []
    for d in data:
        win_position = som.winner(d)
        if win_position in winmap:
            result.append(winmap[win_position].most_common()[0][0])
        else:
            result.append(default_class)
    return result

In [16]:
# set hyperparameters
n_neurons = 20
m_neurons = 20
input_len = X_train.shape[1]
sigma = 1.5
learning_rate = 1.2
iterations = 10000

In [17]:
from minisom import MiniSom
# initialization
som = MiniSom(x=n_neurons, y=m_neurons,input_len=input_len,sigma=sigma,learning_rate=learning_rate)


In [ ]:
%%time
som.train(X_train, iterations, verbose=True)  # random training

In [19]:
%%time
y_pred = classify(som, X_test)
report = classification_report(y_test, y_pred, digits=4)
print(report)

/users/mik2002/.local/lib/python3.10/site-packages/sklearn/utils/_array_api.py:390: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y_pred contains NaN.